In [ ]:
import numpy as np
import simpy

from scipy.stats import gamma

## Problem

A wrestling tournament has 10 weight classes (we'll design them A, B, C,...,J), each with a sixteen competitor bracket - the tournament is single elimination, with four rounds. For each weight class: 8 matches in the first round, 4 matches in the second round, 2 matches in the third (semi-final) round, and 1 match in the fourth (final) round.

The matches are all given a number: matches 0-7 are the first round in weight class A, 8-15 are the first round matches in weight class B, ..., 72-79 are the first round in weight class J, 80-83 are the second round in weight class A, 84-87 are the second round in weight class B,..., 116-119 are the second round in weight class J, 120-121 are the third round matches for weight class A, etc (there are 150 matches in total).

There are four mats (we'll call them mats 0, 1, 2, 3). For the purposes of giving competitors time to get ready, each mat has a queue (in terms of match numbers) of length 3 (in addition to the competitors on the mat). Once a match is completed, the first match in the queue for that particular mat goes into "service", while the other two matches in the queue for that mat move up, and the next match numerically that is not currently in a queue for a mat will enter the queue for that mat.

The day starts the following way:

Mat 0: Match 0 (in service), Match 4, 8, 12 in queue
Mat 1: Match 1 (in service), Match 5, 9, 13 in queue
Mat 2: Match 2 (in service), Match 6, 10, 14 in queue
Mat 3: Match 3 (in service), Match 7, 11, 15 in queue

If Match 3 finishes before the other matches, then match 16 will enter the queue for Mat 3, if 2 finishes next, then match 17 will enter the queue for Mat 2, etc. At the end of the day, when there are not enough matches to fill up all the queue, the next match will enter the queue for the mat with the shortest queue length.

The length of time each match (in minutes) follows a gamma distribution with shape parameter $3$ and rate parameter $0.35$.

Run 1000 simulations of the queueing system, and calculate the amount of time until the tournament is complete. Suppose each mat requires 1 referee who is paid USD 60/hour, and a timekeeper, and a scorekeeper who are each paid USD 15/hour. If each entrant to the tournament pays a USD 25 entrance fee (there 160 competitors), how much money does the tournament organizer expect to make or lose? What are the 5th and 95th percentiles of this quantity?

(Note: don't worry about pathological outcomes like say, the final for a division going before one of the semi-finals is complete) 

In [ ]:
def match(shared):
    main_request = shared['main'].request()
    yield main_request
    q_len = [len(shared['mats'][i].queue) + shared['mats'][i].count for i in range(4)]
    ind = np.argmin(q_len)
    rqt = shared['mats'][ind].request()
    yield rqt
    st = gamma.rvs(3, scale=1/0.35)
    yield shared['env'].timeout(st)
    shared['mats'][ind].release(rqt)
    shared['main'].release(main_request)
    shared['end_times'].append(shared['env'].now)


def simulate():
    env = simpy.Environment()
    main = simpy.Resource(env=env, capacity=16)
    mats = [simpy.Resource(env=env, capacity=1) for i in range(4)]
    end_times = []
    shared = {'env': env,
              'main': main,
              'mats': mats,
              'end_times': end_times}
    for i in range(0, 150):
        env.process(match(shared))
    env.run()
    return max(shared['end_times'])


def run_sims(nsims):
    X = np.array([simulate() for _ in range(nsims)])
    cost = 4 * X * (60 + 15 + 15) / 60
    revenue = 25 * 160
    profit = revenue - cost
    return profit

In [ ]:


### output numpy array of simulations here

profits = run_sims(1_000)


### output 5th percentile and 95th percentile

p5 = np.percentile(profits, 5)
p95 = np.percentile(profits,95)
